In [6]:
import ast
import json

import pandas as pd

In [7]:
# convert "['a','b']" to a set {'a', 'b'}
def to_set(x):
    list_string = json.loads(x)
    list_raw = ast.literal_eval(list_string)
    return set(list_raw)

In [8]:
pred_df = pd.read_csv("../data/SpacyProcessing/foodkg_spacy_processed.csv")
# Rename the pred columns for so we don't clash on merge
pred_df = pred_df.rename(
    columns={
        "processed_ingredients": "pred_ingredients",
    }
)
pred_df.head(5)

,recipe_id,original_ingredients,pred_ingredients
0,0,"['1 c. firmly packed brown sugar', '1/2 c. eva...","['brown sugar', 'milk', 'vanilla', 'nut', 'but..."
1,1,"['1 small jar chipped beef, cut up', '4 boned ...","['beef', 'chicken breast', 'cream mushroom sou..."
2,2,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","['corn', 'cream cheese', 'butter', 'garlic', '..."
3,3,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","['chicken', 'chicken gravy', 'cream mushroom s..."
4,4,"['1 c. peanut butter', '3/4 c. graham cracker ...","['peanut butter', 'graham cracker crumb', 'but..."


In [9]:
test_df = pd.read_csv("../data/test/TEST_FoodKG_ingredients_normalized.csv")
# Rename the test columns for consistency, and so we don't clash on merge
test_df = test_df.rename(
    columns={
        "id": "recipe_id",
        "ingredients": "original_ingredients",
        "ingredients_normalized": "ingredients_test"
    })
test_df.head(5)

,recipe_id,original_ingredients,ingredients_test
0,0,"['1 c. firmly packed brown sugar', '1/2 c. eva...","""['brown sugar', 'evaporated milk', 'vanilla',..."
1,1,"['1 small jar chipped beef, cut up', '4 boned ...","""['chipped beef', 'chicken breast', 'cream of ..."
2,2,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","""['corn', 'cream cheese', 'butter', 'garlic po..."
3,3,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","""['whole chicken', 'chicken gravy', 'cream of ..."
4,4,"['1 c. peanut butter', '3/4 c. graham cracker ...","""['peanut butter', 'graham cracker', 'butter',..."


In [10]:
# Merge using test_df as the left df to keep its order
merged_df = test_df[["recipe_id", "ingredients_test"]].merge(
    pred_df, on="recipe_id", how="inner"
)
merged_df.head()

,recipe_id,ingredients_test,original_ingredients,pred_ingredients
0,0,"""['brown sugar', 'evaporated milk', 'vanilla',...","['1 c. firmly packed brown sugar', '1/2 c. eva...","['brown sugar', 'milk', 'vanilla', 'nut', 'but..."
1,1,"""['chipped beef', 'chicken breast', 'cream of ...","['1 small jar chipped beef, cut up', '4 boned ...","['beef', 'chicken breast', 'cream mushroom sou..."
2,2,"""['corn', 'cream cheese', 'butter', 'garlic po...","['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","['corn', 'cream cheese', 'butter', 'garlic', '..."
3,3,"""['whole chicken', 'chicken gravy', 'cream of ...","['1 large whole chicken', '2 (10 1/2 oz.) cans...","['chicken', 'chicken gravy', 'cream mushroom s..."
4,4,"""['peanut butter', 'graham cracker', 'butter',...","['1 c. peanut butter', '3/4 c. graham cracker ...","['peanut butter', 'graham cracker crumb', 'but..."


In [12]:
pred_sets = [set(ast.literal_eval(x)) for x in merged_df["pred_ingredients"]]
test_sets = [to_set(x) for x in merged_df["ingredients_test"]]

new_df = pd.DataFrame({
    "recipe_id": merged_df["recipe_id"],
    "original_ingredients": merged_df["original_ingredients"],
    "pred_set": pred_sets,
    "test_set": test_sets,
})
new_df.head()

,recipe_id,original_ingredients,pred_set,test_set
0,0,"['1 c. firmly packed brown sugar', '1/2 c. eva...","{vanilla, brown sugar, butter margarine, milk,...","{evaporated milk, margarine, vanilla, brown su..."
1,1,"['1 small jar chipped beef, cut up', '4 boned ...","{cream mushroom soup, chicken breast, beef, ca...","{chipped beef, cream of mushroom soup, chicken..."
2,2,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","{butter, garlic, pepper, corn, cream cheese, s...","{butter, garlic powder, corn, black pepper, cr..."
3,3,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","{cream mushroom soup, cheese, chicken gravy, s...","{cheese, chicken gravy, cream of mushroom soup..."
4,4,"['1 c. peanut butter', '3/4 c. graham cracker ...","{graham cracker crumb, powdered sugar, chocola...","{graham cracker, powdered sugar, chocolate chi..."


In [13]:
def precision_recall_f1_per_row(row):
    pred = row["pred_set"]
    test = row["test_set"]

    tp = len(pred & test)
    fp = len(pred - test)
    fn = len(test - pred)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    return pd.Series({"precision": precision, "recall": recall, "f1": f1})


def calculate_overlap(row):
    intersection_size = len(row["pred_set"] & row["test_set"])
    n = max(len(row["pred_set"]), len(row["test_set"]))
    if n == 0: return 0
    return intersection_size / n


def calculate_jaccard(row):
    intersection_size = row['intersection_size']
    union_size = row['union_size']
    if union_size == 0: return 0
    return intersection_size / union_size

In [14]:
# Elements that differ
new_df["pred_only"] = new_df["pred_set"] - new_df["test_set"]
new_df["test_only"] = new_df["test_set"] - new_df["pred_set"]
# intersection size
new_df["intersection_size"] = new_df.apply(
    lambda row: len(row["pred_set"] & row["test_set"]), axis=1
)
# union size
new_df["union_size"] = new_df.apply(
    lambda row: len(row["pred_set"] | row["test_set"]), axis=1
)
# overlap = intersection / larger_set
new_df["overlap"] = new_df.apply(calculate_overlap, axis=1)
# Jaccard similarity (intersection / union)
new_df["jaccard"] = new_df.apply(calculate_jaccard, axis=1)
# Precision, recall, F1
new_df[["precision", "recall", "f1"]] = new_df.apply(precision_recall_f1_per_row, axis=1)

In [15]:
new_df.head(100)

,recipe_id,original_ingredients,pred_set,test_set,pred_only,test_only,intersection_size,union_size,overlap,jaccard,precision,recall,f1
0,0,"['1 c. firmly packed brown sugar', '1/2 c. eva...","{vanilla, brown sugar, butter margarine, milk,...","{evaporated milk, margarine, vanilla, brown su...","{butter margarine, nut, milk}","{evaporated milk, pecan, margarine, butter}",3,10,0.428571,0.300000,0.500000,0.428571,0.461538
1,1,"['1 small jar chipped beef, cut up', '4 boned ...","{cream mushroom soup, chicken breast, beef, ca...","{chipped beef, cream of mushroom soup, chicken...","{cream mushroom soup, beef, carton sour cream}","{chipped beef, cream of mushroom soup, sour cr...",1,7,0.250000,0.142857,0.250000,0.250000,0.250000
2,2,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","{butter, garlic, pepper, corn, cream cheese, s...","{butter, garlic powder, corn, black pepper, cr...","{garlic, pepper}","{black pepper, garlic powder}",4,8,0.666667,0.500000,0.666667,0.666667,0.666667
3,3,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","{cream mushroom soup, cheese, chicken gravy, s...","{cheese, chicken gravy, cream of mushroom soup...","{cream mushroom soup, chicken}","{cream of mushroom soup, whole chicken}",3,7,0.600000,0.428571,0.600000,0.600000,0.600000
4,4,"['1 c. peanut butter', '3/4 c. graham cracker ...","{graham cracker crumb, powdered sugar, chocola...","{graham cracker, powdered sugar, chocolate chi...",{graham cracker crumb},{graham cracker},4,6,0.800000,0.666667,0.800000,0.800000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,"['3 (1 lb.) cans pork and beans', '1/2 c. bell...","{pork bean, onion, brown sugar, bell pepper, o...","{onion, brown sugar, bell pepper, ground beef,...","{pork bean, beef}","{pork and beans, ground beef}",7,11,0.777778,0.636364,0.777778,0.777778,0.777778
96,96,"['2 c. diced cooked chicken', '2 Tbsp. shorten...","{water, carrot, onion, shortening, brown sugar...","{water, carrot, onion, shortening, brown sugar...","{chicken bouillon cube, soy}","{soy sauce, chicken bouillon}",12,16,0.857143,0.750000,0.857143,0.857143,0.857143
97,97,"['8 to 10 juicy oranges, peeled and diced', '1...","{cherry, moist coconut, sugar, juicy orange, o...","{cherry, coconut, sugar, orange, orange juice,...","{juicy orange, moist coconut}","{orange, coconut}",4,8,0.666667,0.500000,0.666667,0.666667,0.666667
98,98,"['1 c. creamy peanut butter', '1 c. sugar', '1...","{sugar, egg, creamy peanut butter}","{sugar, egg, peanut butter}",{creamy peanut butter},{peanut butter},2,4,0.666667,0.500000,0.666667,0.666667,0.666667


In [16]:
print(f"Overlap:   {new_df['overlap'].mean():.4f} (Intersection divided by the size of the larger set)")
print(f"Jaccard:   {new_df['jaccard'].mean():.4f} (Intersection divided by the size of the union)")
print(f"Precision: {new_df['precision'].mean():.4f} (Fraction of predicted ingredients that are correct)")
print(f"Recall:    {new_df['recall'].mean():.4f} (Fraction of actual ingredients that were predicted)")
print(f"F1:        {new_df['f1'].mean():.4f} (Harmonic mean of precision and recall)")

Overlap:   0.6933 (Intersection divided by the size of the larger set)
Jaccard:   0.5908 (Intersection divided by the size of the union)
Precision: 0.7214 (Fraction of predicted ingredients that are correct)
Recall:    0.7027 (Fraction of actual ingredients that were predicted)
F1:        0.7096 (Harmonic mean of precision and recall)
